# 1D Fractional Pixel Spectral Extraction from NIRSpec IFU cube

This notebook shows how to do fractional pixel extraction from a spectral IFU cube with variable extraction aperture as a function of wavelength.

In [ ]:
from astropy import units as u
from astropy import wcs
from astropy.io import fits
from astropy.nddata import StdDevUncertainty
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import SkyCircularAperture, aperture_photometry
from regions import read_ds9, CircleSkyRegion
from specutils import Spectrum1D
import glue_jupyter as gj
import matplotlib
from matplotlib import pyplot as plt
import numpy as np

%matplotlib inline

### Grab the NIRSpec IFU cube data from Box

In [ ]:
from urllib import request
import os
import tempfile

url_base = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/cube_fitting/Q3D_20200407/"
filename = "Q3D_NRS_491_s3d.fits"
url = f"{url_base}{filename}"

tmpdir = tempfile.gettempdir()
filename = os.path.join(tmpdir, filename)

request.urlretrieve(url, filename=filename)
print(filename)

### Load the IFU cube into glue-jupyter viewer

In [ ]:
app = gj.jglue()
app.load_data(filename)
data_sci = "Q3D_NRS_491_s3d[SCI]"
image_viewer = app.imshow(data=data_sci)

**Stop!** Now select a circular region in the viewer above for the extraction aperture.  Then continue.

Convert selected area above to an astropy region in pixel coordinates

In [ ]:
data = app.data_collection[data_sci]
pix_region = data.get_selection_definition(format='astropy-regions')
print(pix_region)

### Create a `SkyCircularAperture`

In [ ]:
# Get the sci, err and wcs from the file
with fits.open(filename, memmap=False) as hdulist:
    sci = hdulist["SCI"].data
    err = hdulist["ERR"].data
    w = wcs.WCS(hdulist[1].header)

# Convert CirclePixelRegion to CircleSkyRegion
pixel_scale = np.sqrt(np.abs(np.prod(w.celestial.wcs.cdelt)))
center = w.celestial.pixel_to_world(pix_region.center.x, pix_region.center.y)
radius = pix_region.radius * pixel_scale * 3600 * u.arcsec
region = CircleSkyRegion(center, radius)
aperture = SkyCircularAperture(region.center, region.radius)
print(aperture)

Plot up our SkyCircularAperture and verify it reflects our selected region above in glue

In [ ]:
pixel_region = region.to_pixel(w.celestial)
ax = plt.subplots()[1]
norm = ImageNormalize(stretch=SqrtStretch())
ax.imshow(np.mean(sci, axis=0), cmap='gray', origin='lower', norm=norm)
pixel_region.plot(axes=ax)

### Now for each spatial slice, compute the sum in the aperture

In [ ]:
# Read cube into Spectrum1D in order to do 1D extraction
spec1d = Spectrum1D.read(filename)

flux_sum = []
err_sum = []

# We will scale the aperture size linearly by wavelength.  Define the reference wavelength.
reference_wavelength = spec1d.spectral_axis[0]

for wavelength, sci_slice, err_slice in zip(spec1d.spectral_axis, sci, err):
    aperture_radius = aperture.r * wavelength / reference_wavelength
    aperture_cone = SkyCircularAperture(region.center, aperture_radius)
    phot_table = aperture_photometry(sci_slice, aperture_cone, wcs=w.celestial,
                                     method="exact", error=err_slice)
    flux_sum.append(phot_table["aperture_sum"][0])
    err_sum.append(phot_table["aperture_sum_err"][0])

flux = np.array(flux_sum) * spec1d.flux.unit
uncertainty = StdDevUncertainty(np.array(err_sum))

In [ ]:
# Put the extracted flux and uncertainty into a new Spectrum1D object
extracted_spec = Spectrum1D(flux=flux, spectral_axis=spec1d.spectral_axis,
                            uncertainty=uncertainty)

### Plot it up

In [ ]:
ax = plt.subplots()[1]
ax.plot(extracted_spec.spectral_axis, extracted_spec.flux)
ax.set_xlim(0.95,1.45)
ax.set_ylim(0,50000)
ax.set_xlabel(f"Dispersion ({extracted_spec.spectral_axis.unit.to_string()})")
ax.set_ylabel(f"Flux ({extracted_spec.flux.unit.to_string()})")
plt.plot()